In [1]:
import diff_classifier.aws as aws
import diff_classifier.utils as ut
import diff_classifier.msd as msd
import diff_classifier.features as ft

import numpy as np
import pandas as pd

In [2]:
import pandas as pd
import numpy as np
import skimage.io as sio
import numpy.ma as ma
import pandas.util.testing as pdt
import numpy.testing as npt
import diff_classifier.msd as msd

In [ ]:
folder = 'Gel_Studies/08_14_18_gel_validation'
file = 'Traj_100nm_XY01_0_1.csv'

#aws.download_s3('{}/{}'.format(folder, file), file, bucket_name='ccurtis.data')
Traj = ut.csv_to_pd(file)

In [3]:
data1 = {'Frame': [0, 1, 2, 3, 4, 0, 1, 2, 3, 4],
         'Track_ID': [1, 1, 1, 1, 1, 2, 2, 2, 2, 2],
         'X': [5, 6, 7, 8, 9, 1, 2, 3, 4, 5],
         'Y': [6, 7, 8, 9, 10, 2, 3, 4, 5, 6],
         'Quality': [10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
         'SN_Ratio': [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
         'Mean_Intensity': [10, 10, 10, 10, 10, 10, 10, 10, 10, 10]}
df = pd.DataFrame(data=data1)

In [4]:
length = max(df['Frame']) + 1
testmsd = msd.all_msds2(df, frames=length)
testft = ft.calculate_features(testmsd, frame=3)

In [7]:
testft

,AR,D_fit,Deff,MSD_ratio,Mean_Intensity,Quality,SN_Ratio,Track_ID,X,Y,...,asymmetry2,asymmetry3,boundedness,efficiency,elongation,fractal_dim,frames,kurtosis,straightness,trappedness
0,6.369052e+15,0.5,1.5,-0.222222,10.0,10.0,0.1,1.0,7.0,8.0,...,1.110223e-16,0.693147,0.25,4.0,1.0,1.0,5.0,1.259066,1.0,-0.152585
1,1.698414e+16,0.5,1.5,-0.222222,10.0,10.0,0.1,2.0,3.0,4.0,...,1.110223e-16,0.693147,0.25,4.0,1.0,1.0,5.0,2.283058,1.0,-0.152585


In [ ]:
np.nanmean(testdata[testdata['Track_ID']==1]['Mean_Intensity'].values)

In [ ]:
try: testdata[testdata['Track_ID']==1]['MSDs'][7]
except: KeyError: print('NaN')

In [ ]:
def calculate_features(dframe, framerate=1, frame=100):
    """Calculates multiple features from input MSD dataset and stores in pandas
    dataframe.

    Parameters
    ----------
    dframe : pandas.core.frame.DataFrame
        Output from msd.all_msds2.  Must have at a minimum the following
        columns:
        Track_ID, Frame, X, Y, and MSDs.
    framerate : int or float
        Framerate of the input videos from which trajectories were calculated.
        Required for accurate calculation of some features.  Default is 1.
        Possibly not required. Ignore if performing all calcuations without
        units.
    frame : int
        Frame at which to calculate Deff

    Returns
    -------
    datai: pandas.core.frame.DataFrame
        Contains a row for each trajectory in dframe.  Holds the following
        features of each trajetory: Track_ID, alpha, D_fit, kurtosis,
        asymmetry1, asymmetry2, asymmetry3, aspect ratio (AR), elongation,
        boundedness, fractal dimension (fractal_dim), trappedness, efficiency,
        straightness, MSD ratio, frames, X, and Y.

    Examples
    --------
    See example outputs from individual feature functions.

    """

    # Skeleton of Trajectory features metadata table.
    # Builds entry for each unique Track ID.
    holder = dframe.Track_ID.unique().astype(float)
    die = {'Track_ID': holder,
           'alpha': holder,
           'D_fit': holder,
           'kurtosis': holder,
           'asymmetry1': holder,
           'asymmetry2': holder,
           'asymmetry3': holder,
           'AR': holder,
           'elongation': holder,
           'boundedness': holder,
           'fractal_dim': holder,
           'trappedness': holder,
           'efficiency': holder,
           'straightness': holder,
           'MSD_ratio': holder,
           'frames': holder,
           'X': holder,
           'Y': holder,
           'Quality': holder,
           'Mean_Intensity': holder,
           'SN_Ratio': holder,
           'Deff': holder}

    datai = pd.DataFrame(data=die)

    trackids = dframe.Track_ID.unique()
    partcount = trackids.shape[0]

    for particle in range(0, partcount):
        single_track_masked =\
         dframe.loc[dframe['Track_ID'] ==
                    trackids[particle]].sort_values(['Track_ID', 'Frame'],
                                                    ascending=[
                                                    1,
                                                    1]).reset_index(drop=True)
        single_track = unmask_track(single_track_masked)
        (datai['alpha'][particle],
         datai['D_fit'][particle]) = alpha_calc(single_track)
        datai['kurtosis'][particle] = kurtosis(single_track)
        (eig1, eig2, datai['asymmetry1'][particle],
         datai['asymmetry2'][particle],
         datai['asymmetry3'][particle]) = asymmetry(single_track)
        (datai['AR'][particle], datai['elongation'][particle],
         (datai['X'][particle],
          datai['Y'][particle])) = aspectratio(single_track)
        (datai['boundedness'][particle], datai['fractal_dim'][particle],
         datai['trappedness'][particle]) = boundedness(single_track, framerate)
        (datai['efficiency'][particle],
         datai['straightness'][particle]) = efficiency(single_track)
        datai['frames'][particle] = single_track.shape[0]
        if single_track['Frame'][single_track.shape[0]-2] > 2:
            datai['MSD_ratio'][particle] = msd_ratio(single_track, 2,
                                                     single_track['Frame'][
                                                      single_track.shape[0]-2])
        else:
            datai['MSD_ratio'][particle] = np.nan

        try: datai['Deff'][particle] = single_track['MSDs'][frame] / (4*frame)
        except datai['Deff'][particle] = np.nan
        
        datai['Mean_Intensity'] = np.nanmean(single_track['Mean_Intensity'].values)
        datai['Quality'] = np.nanmean(single_track['Quality'].values)
        datai['SN_Ratio'] = np.nanmean(single_track['SN_Ratio'].values)

    return datai